Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
train_dataset_original = train_dataset
train_labels_original = train_labels
valid_dataset_original = valid_dataset
valid_labels_original = valid_labels
test_dataset_original = test_dataset
test_labels_original = test_labels

In [7]:
# Logistic Model

batch_size = 128

def logistic_model_train_test(batch_size, beta=1e-02, num_steps=3001):
    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
      # regularization
      loss = loss + beta*tf.nn.l2_loss(weights)

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

    ###############################################################################

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(graph=graph, config=config) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [8]:
logistic_model_train_test(batch_size, 1e-02)

Initialized
Minibatch loss at step 0: 48.595909
Minibatch accuracy: 7.8%
Validation accuracy: 11.6%
Minibatch loss at step 500: 1.167122
Minibatch accuracy: 69.5%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 0.808150
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 0.725581
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.760352
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.804050
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.849650
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
Test accuracy: 87.4%


Test accuracy with regularization ~ 87.4%

Test accuracy without regularization ~ 84.6% (from assignment 2)

In [9]:
# Neural network model

batch_size = 128
hidden_nodes = 1024

def graph_computation(inputs, weights_1, biases_1, weights_2, biases_2, labels=None, dropout_keep_pb=1):
    first_layer_out = tf.matmul(inputs, weights_1) + biases_1
    relu_out = tf.nn.relu(first_layer_out)
    # use dropout: https://www.tensorflow.org/get_started/mnist/pros
    # if dropout_keep_pb == 1, all elements in Tensor are kept, otherwise dropout is applied
    dropout_out = tf.nn.dropout(relu_out, dropout_keep_pb)
    logits = tf.matmul(dropout_out, weights_2) + biases_2
    if labels is not None:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        return logits, loss
    return logits, _

def neural_network_model_train_test(batch_size, hidden_nodes, beta, keep_training_prob=1, num_steps=3001):
    graph = tf.Graph()
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
        
      #dropout prob
      keep_prob = tf.placeholder(tf.float32)

      # Variables.
      weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
      biases_1 = tf.Variable(tf.zeros([hidden_nodes]))

      weights_2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
      biases_2 = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits, loss = graph_computation(tf_train_dataset, 
                                       weights_1, biases_1, weights_2, biases_2, 
                                       labels=tf_train_labels, dropout_keep_pb=keep_prob)
      # regularization (if beta==0 we have no regularization term) 
      loss = loss + beta*tf.nn.l2_loss(weights_1) + beta*tf.nn.l2_loss(weights_2)

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      valid_logits, _ = graph_computation(tf_valid_dataset, 
                                          weights_1, biases_1, weights_2, biases_2)
      valid_prediction = tf.nn.softmax(valid_logits)

      test_logits, _ = graph_computation(tf_test_dataset, 
                                          weights_1, biases_1, weights_2, biases_2)

      test_prediction = tf.nn.softmax(test_logits)


    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(graph=graph, config=config) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : keep_training_prob}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [10]:
neural_network_model_train_test(batch_size, hidden_nodes, 1e-03)

Initialized
Minibatch loss at step 0: 693.865845
Minibatch accuracy: 4.7%
Validation accuracy: 34.9%
Minibatch loss at step 500: 204.940887
Minibatch accuracy: 74.2%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 113.935837
Minibatch accuracy: 85.9%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 68.739326
Minibatch accuracy: 85.2%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 41.417973
Minibatch accuracy: 82.8%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 25.409416
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.513600
Minibatch accuracy: 83.6%
Validation accuracy: 86.8%
Test accuracy: 92.5%


Test accuracy with regularization ~ 92.7%

Test accuracy without regularization ~ 87% (from assignment 2)

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

Note the difference between Minibatch accuracy and Validation and Testing accuracies

Minibatch accuracies reach 100%, while validation and Testing accuracies drops to ~60%, ~70%

And Test accuracies drop respect to using the whole original dataset (see Problem 1 accuracies)

---

In [11]:
# Restrict your training data to just a few batches
train_dataset = train_dataset_original[:batch_size*2]
train_labels = train_labels_original[:batch_size*2]
valid_dataset = valid_dataset_original[:batch_size*2]
valid_labels = valid_labels_original[:batch_size*2]
test_dataset = test_dataset_original[:batch_size*2]
test_labels = test_labels_original[:batch_size*2]

In [12]:
logistic_model_train_test(batch_size, 1e-02, 3001)

Initialized
Minibatch loss at step 0: 46.938217
Minibatch accuracy: 9.4%
Validation accuracy: 14.1%
Minibatch loss at step 500: 0.311061
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 1000: 0.120612
Minibatch accuracy: 100.0%
Validation accuracy: 65.6%
Minibatch loss at step 1500: 0.119112
Minibatch accuracy: 100.0%
Validation accuracy: 65.2%
Minibatch loss at step 2000: 0.118925
Minibatch accuracy: 100.0%
Validation accuracy: 64.8%
Minibatch loss at step 2500: 0.118798
Minibatch accuracy: 100.0%
Validation accuracy: 64.8%
Minibatch loss at step 3000: 0.118707
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Test accuracy: 82.8%


In [13]:
neural_network_model_train_test(batch_size, hidden_nodes, 1e-03)

Initialized
Minibatch loss at step 0: 632.961975
Minibatch accuracy: 7.8%
Validation accuracy: 31.2%
Minibatch loss at step 500: 190.363312
Minibatch accuracy: 100.0%
Validation accuracy: 62.1%
Minibatch loss at step 1000: 115.446709
Minibatch accuracy: 100.0%
Validation accuracy: 62.1%
Minibatch loss at step 1500: 70.013214
Minibatch accuracy: 100.0%
Validation accuracy: 62.1%
Minibatch loss at step 2000: 42.459877
Minibatch accuracy: 100.0%
Validation accuracy: 62.1%
Minibatch loss at step 2500: 25.749996
Minibatch accuracy: 100.0%
Validation accuracy: 61.7%
Minibatch loss at step 3000: 15.616266
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Test accuracy: 73.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

If we use Dropout while removing L2 regularization, the network keeps overfitting, although not as much as the case with no dropout but L2 regularization (Training acc vs. Validation acc is 100% vs. ~70.3% ). The test accuracy is better than the case with no dropout but L2 regularization ( ~75.8% vs. ~84.0%). Minibatch loss is 0 from step 500 to the end of training.

If we use Dropout while keeping L2 regularization, the network keeps overfitting (Training acc vs. Validation acc is 100% vs. ~68.0% ). However, Minibatch loss does not reach the zero value in any iteration. Test accuracy is slightly different from the previous case (which applies Dropout and no L2 regularization); sometimes is better than the previous case and sometimes is worse.

---

In [14]:
# Use Dropout without regularization 
neural_network_model_train_test(batch_size, hidden_nodes, 0, keep_training_prob=0.5)

Initialized
Minibatch loss at step 0: 503.593903
Minibatch accuracy: 6.2%
Validation accuracy: 37.5%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 65.6%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Test accuracy: 86.3%


In [15]:
# Use Dropout with regularization 
neural_network_model_train_test(batch_size, hidden_nodes, 1e-03, keep_training_prob=0.5)

Initialized
Minibatch loss at step 0: 836.228943
Minibatch accuracy: 7.0%
Validation accuracy: 25.8%
Minibatch loss at step 500: 191.042587
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 1000: 115.875710
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 1500: 70.275673
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 2000: 42.620182
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 2500: 25.847380
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 3000: 15.676021
Minibatch accuracy: 100.0%
Validation accuracy: 67.2%
Test accuracy: 82.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [16]:
# Restore original data
train_dataset = train_dataset_original
train_labels = train_labels_original
valid_dataset = valid_dataset_original
valid_labels = valid_labels_original
test_dataset = test_dataset_original
test_labels = test_labels_original

In [41]:
# Neural network model with learning rate decay and a second hidden layer

batch_size = 128
hidden_nodes = 1024

def graph_computation_layer1(inputs, weights_1, biases_1):
    first_layer_out = tf.matmul(inputs, weights_1) + biases_1
    relu_out = tf.nn.relu(first_layer_out)
    return relu_out

def graph_computation_layer2(inputs, weights_1, biases_1, weights_2, biases_2, labels=None, dropout_keep_pb=1):
    first_layer_out = tf.matmul(inputs, weights_1) + biases_1
    relu_out = tf.nn.relu(first_layer_out)
    # use dropout: https://www.tensorflow.org/get_started/mnist/pros
    # if dropout_keep_pb == 1, all elements in Tensor are kept, otherwise dropout is applied
    dropout_out = tf.nn.dropout(relu_out, dropout_keep_pb)
    logits = tf.matmul(dropout_out, weights_2) + biases_2
    if labels is not None:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        return logits, loss
    return logits, _

def neural_network_model2_train_test(batch_size, hidden_nodes, beta, keep_training_prob=1, num_steps=3001, lr0=None):
    graph = tf.Graph()
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
        
      #dropout prob
      keep_prob = tf.placeholder(tf.float32)

      # Variables.
      weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
      biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
      hidden_nodes2 = int(hidden_nodes/4)
        
      weights_2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, hidden_nodes2]))
      biases_2 = tf.Variable(tf.zeros([hidden_nodes2]))

      weights_3 = tf.Variable(
        tf.truncated_normal([hidden_nodes2, num_labels]))
      biases_3 = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      first_layer_out = graph_computation_layer1(tf_train_dataset, weights_1, biases_1)
      logits, loss = graph_computation_layer2(first_layer_out, 
                                       weights_2, biases_2, weights_3, biases_3, 
                                       labels=tf_train_labels, dropout_keep_pb=keep_prob)
      # regularization (if beta==0 we have no regularization term) 
      loss = loss + beta*tf.nn.l2_loss(weights_1) + beta*tf.nn.l2_loss(weights_2) + 1e-02*beta*tf.nn.l2_loss(weights_3)

      # Optimizer.
      if lr0 == None:
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      else:
        global_step = tf.Variable(0)  # count the number of steps taken
        learning_rate = tf.train.exponential_decay(lr0, global_step, 1000, 0.95)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      
      partial_computation_v = graph_computation_layer1(tf_valid_dataset, weights_1, biases_1)
      valid_logits, _ = graph_computation_layer2(partial_computation_v, 
                                          weights_2, biases_2, weights_3, biases_3)
      valid_prediction = tf.nn.softmax(valid_logits)

      partial_computation = graph_computation_layer1(tf_test_dataset, weights_1, biases_1)
      test_logits, _ = graph_computation_layer2(partial_computation, 
                                          weights_2, biases_2, weights_3, biases_3)

      test_prediction = tf.nn.softmax(test_logits)


    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(graph=graph, config=config) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : keep_training_prob}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [46]:
# Additional hidden layer
neural_network_model2_train_test(batch_size, hidden_nodes, 1e-02, keep_training_prob=1, num_steps=20001, lr0=0.025)

Initialized
Minibatch loss at step 0: 8436.404297
Minibatch accuracy: 12.5%
Validation accuracy: 26.1%
Minibatch loss at step 1000: 2536.013184
Minibatch accuracy: 47.7%
Validation accuracy: 45.1%
Minibatch loss at step 2000: 1596.523438
Minibatch accuracy: 50.8%
Validation accuracy: 56.0%
Minibatch loss at step 3000: 1028.642212
Minibatch accuracy: 59.4%
Validation accuracy: 62.8%
Minibatch loss at step 4000: 677.669434
Minibatch accuracy: 63.3%
Validation accuracy: 68.2%
Minibatch loss at step 5000: 455.641357
Minibatch accuracy: 72.7%
Validation accuracy: 71.3%
Minibatch loss at step 6000: 312.617737
Minibatch accuracy: 76.6%
Validation accuracy: 74.0%
Minibatch loss at step 7000: 218.988831
Minibatch accuracy: 73.4%
Validation accuracy: 76.7%
Minibatch loss at step 8000: 155.793030
Minibatch accuracy: 80.5%
Validation accuracy: 79.0%
Minibatch loss at step 9000: 113.146210
Minibatch accuracy: 78.9%
Validation accuracy: 80.8%
Minibatch loss at step 10000: 83.425056
Minibatch accurac

### Conclusion Problem 4

Need to test different values for neural_network_model2_train_test input arguments, and also for tf.train.exponential_decay

Deeper net --> more training steps, less learning rate, less beta (regularization parameter)


tf.train.exponential_decay(lr0, global_step, 1000, 0.90, staircase=True)

hidden nodes layer 2: 512

beta=1e-04, keep_training_prob=0.5, num_steps=10000, lr0=0.075 --> Test acc: 29.3%

beta=1e-04, keep_training_prob=0.5, num_steps=10000, lr0=0.055 --> Test acc: 45.3%

beta=1e-04, keep_training_prob=1, num_steps=10000, lr0=0.055 --> Test acc: 60.1%

beta=1e-05, keep_training_prob=1, num_steps=10000, lr0=0.055 --> Test acc: 72.6%

beta=1e-05, keep_training_prob=1, num_steps=10000, lr0=0.025 --> Test acc: 75.0%

beta=1e-06, keep_training_prob=1, num_steps=10000, lr0=0.025 --> Test acc: 76.5%

beta=1e-06, keep_training_prob=1, num_steps=10000, lr0=0.01 --> Test acc: 75.5%

hidden nodes layer 2: 256

beta=1e-06, keep_training_prob=1, num_steps=10000, lr0=0.01 --> Test acc: 73.8%

beta=1e-07, keep_training_prob=1, num_steps=10000, lr0=0.01 --> Test acc: 74.0%

beta=1e-07, keep_training_prob=1, num_steps=10000, lr0=0.005 --> Test acc: 76.2%

beta=1e-07, keep_training_prob=1, num_steps=10000, lr0=0.001 --> Test acc: 84.0%

beta=1e-07, keep_training_prob=1, num_steps=20000, lr0=0.001 --> Test acc: 83.5%

beta=1e-07, keep_training_prob=1, num_steps=20000, lr0=0.0005 --> Test acc: 84.5%

beta=1e-06, keep_training_prob=1, num_steps=10000, lr0=0.001 --> Test acc: 83%

change exponential decay: 

tf.train.exponential_decay(lr0, global_step, 1000, 0.65)

beta=1e-06, keep_training_prob=1, num_steps=10000, lr0=0.001 --> Test acc: 84.7%

beta=1e-06, keep_training_prob=1, num_steps=10000, lr0=0.01 --> Test acc: 67%

change betas: multiply beta3 times 1e-02

beta=1e-02, keep_training_prob=1, num_steps=10000, lr0=0.001 --> Test acc: 85.1%

tf.train.exponential_decay(lr0, global_step, 1000, 0.95)

beta=1e-02, keep_training_prob=1, num_steps=10001, lr0=0.001 --> Test acc: 83.4%

beta=1e-02, keep_training_prob=1, num_steps=10001, lr0=0.01 --> Test acc: 82.8%

beta=1e-02, keep_training_prob=1, num_steps=15001, lr0=0.01 --> Test acc: 85.8%

beta=1e-02, keep_training_prob=1, num_steps=20001, lr0=0.01 --> Test acc: 87.6%

beta=1e-02, keep_training_prob=1, num_steps=20001, lr0=0.025 --> Test acc: 92.1%